In [43]:
#our guide
#https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/TimeSliderChoropleth.ipynb

In [44]:
# this cell likely redundant but keeping until can be confirmed
import geopandas as gpd

assert "naturalearth_lowres" in gpd.datasets.available
datapath = gpd.datasets.get_path("naturalearth_lowres")
gdf = gpd.read_file(datapath)

In [45]:
import pandas as pd

df_all_data = pd.read_csv('srdb-data-V5.csv')[['Latitude', 'Longitude', 'Study_midyear', 'Study_number', 'Rs_annual']].dropna().astype(str)

/home/digiadmin/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,39,61) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


build our time index so that we can put everything in date order later

In [60]:
from datetime import datetime

date_index = df_all_data['Study_midyear'].unique()

for index, dt in enumerate(date_index):
    dt = datetime.strptime(dt[:4] + '-06-01', '%Y-%m-%d').timestamp()
    date_index[index] = dt

date_index.sort()
#date_index

In [67]:
styledata = {}

for study_id in df_all_data['Study_number'].unique():
    time_series = {}
    for study_data in df_all_data[df_all_data['Study_number']==study_id
                                ].set_index(['Study_number']).values:
        dt_time = study_data[2][:4] + '-06-01'
        dt_obj = datetime.strptime(dt_time, '%Y-%m-%d')
        millisec = dt_obj.timestamp()
        dt_index = str(millisec)
        time_series[dt_index] = study_data[3]
    # fails at code below - ValueError: Shape of passed values is (1, 1), indices imply (107, 1)
    study_data = pd.DataFrame(
        {
            'test'
        },
        index=date_index,
    )
    styledata[study_id] = study_data
print(styledata)

{'991350000.0': '375.0'}


ValueError: Shape of passed values is (1, 1), indices imply (107, 1)